# Particle Beams

This example describes all of the major capabilities of particle beams in OSCARS.  There are pre-defined particle types and the user is free to describe any other particle type they wish.  Non-zero beam widths can be defined using th emittance, beta functions, and lattice reference (point at which the beta functions are defined).  One can also define multiple particle beams in one OSCARS object and give weights to each.

Patricle beams can as well be defined, like other OSCARS objects, in any direction.  Although examples are often given where $+\hat z$ is the beam direction (mostly to align with accelerator nomenclature), there is no preferred direction in OSCARS.

The elements which you must define for a beam at a minimum are:
* type  (one of the predefined types, or 'custom')
* name  (any unique name you like)
* energy_GeV  (the energy in GeV)
* x0  (position where initial conditions are given)
* d0  (direction of beam at initial position)
* current  (the current in [A])

For a non-zero emittance beam you must additionally define the following (the beam shape is taken to be gaussian, though other distributions are possible):
* sigma_energy_GeV  (width in energy spread in [GeV])
* emittance  (the beam emittance in [m.rad] as [h, v])
* beta  (the beta function in [m] as [h, v])
* lattice_reference  (the reference point at which beta is defined)
* horizontal_direction  (the horizontal direction of the beam)

For a particle beam you can additionally define the following:
* rotations  (rotate the beam about x, then y, then z given as [$\theta_x, \theta_y, \theta_z$])
* translation  (translate the beam in x, y, z given as [x, y, z])
* t0  (time at the position x0 defined above, default is 0)
* weight  (for randome particle generation with multiple beams, default is 1)

Custom:  If you set type='custom' you will need to also define the following:
* mass (in [kg])
* charge (in [C])

In [ ]:
# matplotlib plots inline
%matplotlib inline

# Import the OSCARS SR module
import oscars.sr

# Import OSCARS plots (matplotlib)
from oscars.plots_mpl import *

In [ ]:
# Create an OSCARS SR object
osr = oscars.sr.sr()

# For these examples we will make use of a simple undulator field
osr.add_bfield_undulator(bfield=[0, 1, 0], period=[0, 0, 0.100], nperiods=5)

plot_bfield(osr)

## Basic  beams /  Particles

These are basic beams with zero emittance, one electron (NSLSII energy) and one proton (LHC energy)

In [ ]:
# Add a basic electron beam with zero emittance
osr.set_particle_beam(type='electron', name='beam_0', energy_GeV=3, x0=[0, 0, -1], d0=[0, 0, 1], current=0.500)

# You MUST set the start and stop time for the calculation
osr.set_ctstartstop(0, 2)

# Plot trajectory of beam
plot_trajectory_position(osr.calculate_trajectory())

In [ ]:
# Add a basic proton beam with zero emittance
osr.set_particle_beam(type='proton', name='beam_1', energy_GeV=13, x0=[0, 0, -1], d0=[0, 0, 1], current=0.500)

# You MUST set the start and stop time for the calculation
osr.set_ctstartstop(0, 2)

# Plot trajectory of beam
plot_trajectory_position(osr.calculate_trajectory())

## Multiple beams

This examples defined multiple beams in the same object, then plots the trajectory for two random particles selected from these beams.

Instead of using set_particle_beam we will use add_particle_beam (which is really the same thing, but set_particle_beam issues a clear_particle_beams() internally)

In [ ]:
# Add basic electron beams with zero emittance
osr.clear_particle_beams()
osr.add_particle_beam(type='electron', name='beam_0', energy_GeV=3,  x0=[0, 0, -1], d0=[0, 0, 1], current=0.500)
osr.add_particle_beam(type='positron', name='beam_1', energy_GeV=3,  x0=[0, 0, -1], d0=[0, 0, 1], current=0.500)
osr.add_particle_beam(type='proton',   name='beam_2', energy_GeV=13, x0=[0, 0, -1], d0=[0, 0, 1], current=0.500)
osr.add_particle_beam(type='muon',     name='beam_3', energy_GeV=2,  x0=[0, 0, -1], d0=[0, 0, 1], current=0.500)
osr.add_particle_beam(type='pi+',      name='beam_4', energy_GeV=1,  x0=[0, 0, -1], d0=[0, 0, 1], current=0.500)

# You MUST set the start and stop time for the calculation
osr.set_ctstartstop(0, 2)

# Plot trajectory of beam
plot_trajectory_position(osr.calculate_trajectory())
osr.set_new_particle()
plot_trajectory_position(osr.calculate_trajectory())
osr.set_new_particle()
plot_trajectory_position(osr.calculate_trajectory())

# Beam with non-zero emittance

Below is an example of a beam with non-zero memittance.  We use the emittance and the beta function.  You must give the reference point for the beta function definition.

In [ ]:
# Add a basic electron beam with zero emittance
osr.set_particle_beam(type='electron',
                      name='beam_0',
                      energy_GeV=3,
                      x0=[0, 0, -1],
                      d0=[0, 0, 1],
                      current=0.500,
                      sigma_energy_GeV=0.001*3,
                      beta=[1.5, 0.8],
                      emittance=[0.9e-9, 0.008e-9],
                      horizontal_direction=[1, 0, 0],
                      lattice_reference=[0, 0, 0])

# You MUST set the start and stop time for the calculation
osr.set_ctstartstop(0, 2)

# Plot trajectory of beam
osr.set_new_particle()
plot_trajectory_position(osr.calculate_trajectory())

In [ ]:
# Let's plot the distribution of particles at the initial position 'position'
energy = []
x = []
y = []
bx = []
by = []
for i in range(5000):
    osr.set_new_particle()
    energy.append(osr.get_particle_e0())
    position = osr.get_particle_x0()
    beta = osr.get_particle_beta0()

    x.append(position[0])
    y.append(position[1])

    bx.append(beta[0])
    by.append(beta[1])
    
plt.figure(1, figsize=[15, 3])

plt.subplot(131)
plt.scatter(x, y)
plt.xlim(-0.0005, 0.0005)
plt.ylim(-0.00005, 0.00005)
plt.xlabel('x0 [m]')
plt.ylabel('y0 [m]')
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))

plt.subplot(132)
plt.scatter(bx, by)
plt.xlim(-0.0002, 0.0002)
plt.ylim(-0.00002, 0.00002)
plt.xlabel('x0 [m]')
plt.ylabel('y0 [m]')
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))

plt.subplot(133)
plt.hist(energy)
plt.xlabel('Energy [GeV]')
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))

plt.show()

## Multiple beams with non-zero emittance

A simple example of 2 beams with different beam parameters in the same object

In [ ]:
# Add basic electron beams with zero emittance
osr.clear_particle_beams()
osr.add_particle_beam(type='electron',
                      name='beam_0',
                      energy_GeV=3,
                      x0=[0, 0, -1],
                      d0=[0, 0, 1],
                      current=0.500,
                      sigma_energy_GeV=0.001*3,
                      beta=[1.5, 0.8],
                      emittance=[0.9e-9, 0.008e-9],
                      horizontal_direction=[1, 0, 0],
                      lattice_reference=[0, 0, 0],
                      weight=0.7)

osr.add_particle_beam(type='electron',
                      name='beam_1',
                      energy_GeV=3.02,
                      x0=[0, 0.000035, -1],
                      d0=[0, 0, 1],
                      current=0.500,
                      sigma_energy_GeV=0.001*3,
                      beta=[1.5, 0.8],
                      emittance=[0.9e-9, 0.008e-9],
                      horizontal_direction=[1, 0, 0],
                      lattice_reference=[0, 0, 0],
                      weight=0.3)


# You MUST set the start and stop time for the calculation
osr.set_ctstartstop(0, 2)

# And plot the results
energy = []
x = []
y = []
for i in range(5000):
    osr.set_new_particle()
    energy.append(osr.get_particle_e0())
    position = osr.get_particle_x0()

    x.append(position[0])
    y.append(position[1])
    
plt.figure(1, figsize=[10, 3])
plt.subplot(121)
plt.scatter(x, y)
plt.xlim(-0.0005, 0.0005)
plt.ylim(-0.00005, 0.00005)
plt.xlabel('x0 [m]')
plt.ylabel('y0 [m]')

plt.subplot(122)
plt.hist(energy)
plt.xlabel('Energy [GeV]')
plt.show()

## Custom Particles

This is an example of a custom particle in OSCARS.  All that additionally needs definition are the charge and mass

In [ ]:
# Add a custom beam with zero emittance.
# This is for a particle with twice the chargs as the electron, but the same mass
osr.set_particle_beam(type='custom',
                      name='beam_1',
                      charge=-2*osr.qe(),
                      mass=osr.me(),
                      energy_GeV=3,
                      x0=[0, 0, -1],
                      d0=[0, 0, 1],
                      current=0.500)

# You MUST set the start and stop time for the calculation
osr.set_ctstartstop(0, 2)

# Plot trajectory of beam
plot_trajectory_position(osr.calculate_trajectory())